In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import sys

import rosbag
import rospy


import sensor_msgs.point_cloud2 as pc2

In [ ]:
MAP_PATH = "ros_map_utils/maps/cancha.pgm"
SEM_MAP_PATH = "ros_map_utils/maps/cancha.png"

RES = 0.05
MAP_WIDTH = 0.5
MAP_HEIGHT = 0.5

In [ ]:
occ_map = cv.imread(MAP_PATH, cv.IMREAD_GRAYSCALE)
seg_map = cv.cvtColor(cv.imread(SEM_MAP_PATH), cv.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(occ_map, cmap='gray')
ax[0].set_title("Occupancy Map")
ax[0].set_xlabel("X (m)")
ax[0].set_ylabel("Y (m)")
ax[1].imshow(seg_map)
ax[1].set_title("Semantic Map")
ax[1].set_xlabel("X (m)")
ax[1].set_ylabel("Y (m)")

print("Occupancy Map Size: ", occ_map.shape)
print("Semantic Map Size: ", seg_map.shape)

In [ ]:
seg_map.max()

In [ ]:
seg_map_gray = cv.cvtColor(seg_map, cv.COLOR_BGR2GRAY)

seg_map_gray[seg_map_gray != 81] = 1
seg_map_gray[seg_map_gray == 81] = 0

plt.imshow(seg_map_gray)

cv.imwrite("ros_map_utils/maps/canchagray.png", seg_map_gray * 255)

In [ ]:
# read a rosbag file

BAG_FILE = "/home/tesistas/Desktop/GONZALO/bags/sec_b_3_2025-05-23-21-32-14.bag"

bag = rosbag.Bag(BAG_FILE, 'r')
bag.get_start_time()

odometry_xy = []
map_info = None
for topic, msg, t in bag.read_messages(topics=['/amcl_pose', "/map"]):
    # clouds.append(np.array(list(pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True))))
    if topic == "/map":
        map_info = msg.info
        # occ_map = np.array(msg.data).reshape((msg.info.height, msg.info.width))
    else:
        odometry_xy.append([msg.pose.pose.position.x, msg.pose.pose.position.y])

bag.close()

odometry_xy = np.array(odometry_xy)
print("Odometry XY: ", odometry_xy.shape)
print("Map Info: ", map_info)

In [ ]:
def world_to_map(x, y):
    """
    Convert world coordinates to map coordinates.
    :param x: x coordinate in world
    :param y: y coordinate in world
    :return: x, y coordinates in map
    """
    x_map = (x - map_info.origin.position.x) / map_info.resolution
    y_map = (y - map_info.origin.position.y) / map_info.resolution
    return x_map, y_map

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.flipud(occ_map), cmap='gray')
ax[0].set_title("Occupancy Map")
ax[0].set_xlabel("X (m)")
ax[0].set_ylabel("Y (m)")
ax[1].imshow(np.flipud(seg_map))
ax[1].set_title("Semantic Map")
ax[1].set_xlabel("X (m)")
ax[1].set_ylabel("Y (m)")

xmap, ymap = world_to_map(odometry_xy[:, 0], odometry_xy[:, 1])
ax[1].plot(xmap, ymap, 'r-', label='Odometry Path')
ax[0].plot(xmap, ymap, 'r-', label='Odometry Path')

ax[1].scatter(xmap[-1], ymap[-1])

print("Occupancy Map Size: ", occ_map.shape)
print("Semantic Map Size: ", seg_map.shape)

In [ ]:
def get_map_slice(map, pose, size_m=15):
    px, py = world_to_map(pose[0], pose[1])
    px, py = int(px), int(py)
    size_px2 = int(size_m / RES)

    mapc = np.flipud(map).copy()
    map_slice = mapc[py-size_px2:py+size_px2, px-size_px2:px+size_px2]
    
    origin = [px, py]

    return map_slice, origin

In [ ]:
def get_path_lenght_interval(odometry_xy, start, lenght=15., N_wpts=15):
    xsq = (odometry_xy[start+1:, 0] - odometry_xy[start:-1, 0]) ** 2
    ysq = (odometry_xy[start+1:, 1] - odometry_xy[start:-1, 1]) ** 2
    distances = np.cumsum(np.sqrt(xsq + ysq), axis=0)

    stop_idx = np.where(distances >= lenght)[0]
    if len(stop_idx) == 0:
        stop_idx = len(odometry_xy) - 1
    else:    
        stop_idx = stop_idx[0] + start
    
    ids = np.linspace(start, stop_idx, N_wpts, dtype=np.int64) # inlcude last point

    return odometry_xy[ids]

In [ ]:
a = np.ones(10)
np.where(a > 10)[0]

In [ ]:
pt = 200

map_slice, ori = get_map_slice(occ_map, odometry_xy[pt])
map_slice_seg, _ = get_map_slice(seg_map, odometry_xy[pt])

print(odometry_xy[pt])
print(ori)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

px, py = world_to_map(odometry_xy[pt][0], odometry_xy[pt][1])

map_drawn = map_slice_seg.copy()

sampled_path = get_path_lenght_interval(odometry_xy, pt)
for (x, y) in sampled_path:
    pxi, pyi = world_to_map(x, y)
    pxi = int(pxi - ori[0] + int(15 / (RES)))
    pyi = int(pyi - ori[1] + int(15 / (RES)))

    map_drawn = cv.circle(map_drawn, (pxi, pyi), 3, (255, 255, 255), -1)


ax[0].imshow(seg_map);
ax[1].imshow(map_drawn)
ax[0].axis('off')
ax[1].axis('off')

## Data organization.

What values do we need in one sample of data?

1. Local map 
2. past 10 values of velocity
3. past 3 values of pointcloud
4. relative Goal (is it relative? nice it actually is)
5. K GT trajectories.


### 1. Local map

In [ ]:
def rotate_image(image, angle, center=None):
  if center is None:
    center = tuple(np.array(image.shape[1::-1]) / 2)
  
  rot_mat = cv.getRotationMatrix2D(center, angle, 1.0)
  result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
  return result, rot_mat


def get_local_map(map, pose, size_m=15, flip=True, color=None):
    px, py = world_to_map(pose[0], pose[1])
    px, py = int(px), int(py)
    size_px2 = int(size_m / RES)

    if flip:
        mapc = np.flipud(map).copy()
    else:
        mapc = map.copy()

    mapc, R = rotate_image(mapc, pose[2] * 180 / np.pi, center=(px, py))

    map_slice = mapc[py-size_px2:py+size_px2, px-size_px2:px+size_px2]
    
    if color is not None:
        map_slice = cv.cvtColor(map_slice, cv.COLOR_BGR2GRAY)
        map_slice[map_slice != color] = 1
        map_slice[map_slice == color] = 0

    origin = [px, py]

    return map_slice, origin, R

### 2. Velocities

In [ ]:
from collections import deque
import random
from cv_bridge import CvBridge

odom_topic = "/panther/odometry/filtered"
scan_topic = "/repub/ouster/points"
amcl_topic = "/amcl_pose"
img_topic  = "/repub/camera/image_raw"

N_VEL = 10  # Number of odom messages
N_LIDAR = 3  # Number of lidar messages

# Load messages from the bag
odom_msgs = []
scan_msgs = []
amcl_msgs = []
img_msgs  = []

with rosbag.Bag(BAG_FILE, 'r') as bag:
    for topic, msg, t in bag.read_messages(topics=[odom_topic, scan_topic, amcl_topic, img_topic]):
        timestamp = t.to_sec()
        if topic == odom_topic:
            odom_msgs.append((timestamp, msg))
        elif topic == scan_topic:
            scan_msgs.append((timestamp, msg))
        elif topic == amcl_topic:
            amcl_msgs.append((timestamp, msg))
        elif topic == img_topic:
            img_msgs.append((timestamp, msg))

In [ ]:
# Select a random AMCL pose
amcl_msgs_lst = np.array([[msg.pose.pose.position.x, msg.pose.pose.position.y] for (t, msg) in amcl_msgs])

ind = np.random.choice(list(range(len(amcl_msgs))))
random_amcl = amcl_msgs[ind]

amcl_time, amcl_msg = random_amcl

# Helper to get last N messages before a given timestamp
def get_last_msgs(msgs, current_time, N):
    return [m for t, m in reversed(msgs) if t <= current_time][:N]

# Get previous K odom and N scan messages
odom_history = get_last_msgs(odom_msgs, amcl_time, N_VEL)
scan_history = get_last_msgs(scan_msgs, amcl_time, N_LIDAR)
img_history  = get_last_msgs(img_msgs, amcl_time, 1)

In [ ]:
def world_to_local(points_world, robot_pose):
    """
    Convert world-frame points to robot-centric local frame.

    :param points_world: Nx2 array of (x, y) points in meters
    :param robot_pose: tuple (x_r, y_r, theta_r)
    :return: Nx2 array of transformed points in local frame
    """
    x_r, y_r, theta_r = robot_pose

    # Translate points so robot is at the origin
    translated = points_world - np.array([x_r, y_r])

    # Rotation matrix to align robot's heading with +Y
    c, s = np.cos(-theta_r), np.sin(-theta_r)
    R = np.array([[c, -s],
                  [s,  c]])

    local_points = translated @ R.T
    return local_points

In [ ]:
from tf.transformations import euler_from_quaternion
bridg = CvBridge()
img = bridg.imgmsg_to_cv2(img_history[0])


q = amcl_msg.pose.pose.orientation
_, _, yaw = euler_from_quaternion([q.x, q.y, q.z, q.w])

pose = [amcl_msg.pose.pose.position.x, amcl_msg.pose.pose.position.y, yaw + np.pi / 2]  # Convert to radians and adjust orientation

local_map, ori, R = get_local_map(seg_map, pose, color=81)

print("Local Map Origin: ", ori)
print("Rotation Matrix:\n ", R)


sampled_path = get_path_lenght_interval(amcl_msgs_lst, ind, lenght=100, N_wpts=20)
local_path = world_to_local(sampled_path, pose)

a = local_map.copy()
for (x, y) in local_path:
    pxi = int(x / RES + int(15 / RES))
    pyi = int(y / RES + int(15 / RES))
    a = cv.circle(a, (pxi, pyi), 2, (1), -1)
    plt.scatter(pxi, pyi, s=2, color='r')

goal = amcl_msgs_lst[-1]
goal_map = world_to_local(goal, pose)

_, goal_rad = process_goal(goal, pose)

print(goal_rad * 180 / np.pi)

line_45 = np.array([(pose[0] + 30) * np.cos(goal_rad), 
                    (pose[1] + 30) * np.sin(goal_rad) ])

line_45_map = world_to_local(line_45, pose)


line_0 = np.array([(pose[0] + 30) * 0 , 
                   (pose[1] + 30) ] * 1)

line_0_map = world_to_local(line_0, pose)



# plt.imshow(img)
plt.imshow(a)

fake = np.array([300, -100])
fake = world_to_local(fake, np.array([0, 0, -np.radians(180)]))
plt.scatter(fake[0], fake[1], color='r')

plt.plot([300, goal_map[0]/RES + 300], [300, goal_map[1]/RES + 300])

plt.plot([300, line_45_map[0]/RES + 300], [300, line_45_map[1]/RES + 300])
plt.plot([300, line_0_map[0]/RES + 300], [300, line_0_map[1]/RES + 300])

### 3. Lidar

In [ ]:
scan_history[0].fields

In [ ]:
# done above i think
pcl = np.array(list(pc2.read_points(scan_history[1], field_names=("x", "y", "z", "intensity", "reflectivity", "ambient"), skip_nans=True)))

# 3d plot of the point cloud
ax = plt.figure().add_subplot(projection='3d')
ax.scatter3D(pcl[:, 0], pcl[:, 1], pcl[:, 2], s=1, c=pcl[:, 5])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud')
plt.show()

In [ ]:
def _process_lidar(points, voxel_size=0.08, max_points=5120):
    """
    Args:
        points: (N, 4) array, where columns are [x, y, z, intensity]
        voxel_size: float
        max_points: int
    Returns:
        (max_points, 4) array with downsampled [x, y, z, intensity] values
    """
    coords = np.floor(points[:, :3] / voxel_size).astype(np.int32)
    _, inv, counts = np.unique(coords, axis=0, return_inverse=True, return_counts=True)

    # Sum xyz and intensity by voxel
    # xyz_intensity = np.concatenate([points[:, :3], points[:, 3]], axis=1)
    sums = np.zeros((counts.shape[0], 4), dtype=np.float32)
    np.add.at(sums, inv, points[:, :4])

    # Divide by counts to get mean per voxel
    means = sums / counts[:, None]

    N = means.shape[0]
    if N > max_points:
        indices = np.random.choice(N, max_points, replace=False)
        means = means[indices]
    elif N < max_points:
        pad = np.zeros((max_points - N, 4), dtype=np.float32)
        means = np.concatenate((means, pad), axis=0)

    return means

In [ ]:
import pickle
with open('/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/bb/1110_0.pkl', 'rb') as f:
    data = pickle.load(f)

pcl = data['lidar'][0]
# pcl = _process_lidar(pcl)
ax = plt.figure().add_subplot(projection='3d')
ax.scatter3D(pcl[:, 0], pcl[:, 1], pcl[:, 2], s=1, c=pcl[:, 3])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud')
plt.show()

In [ ]:
plt.hist((pcl[:, 3] - pcl[:, 3].mean()) / pcl[:, 3].std())


((pcl[:, 3] - pcl[:, 3].mean()) / pcl[:, 3].std()).max()

### 4. Goal

In [ ]:
# use goal
goal = amcl_msgs_lst[-1]

def process_goal(goal, pose, cartesian=False):
    
    if cartesian:
        return goal - pose[:2]
    
    else:
        goal = world_to_local(goal, pose)
        rho = np.linalg.norm( goal )
        theta = np.arctan2(goal[1], goal[0] ) + np.pi / 2

        return np.array([rho, theta])


q = amcl_msg.pose.pose.orientation
_, _, yaw = euler_from_quaternion([q.x, q.y, q.z, q.w])

pose = [amcl_msg.pose.pose.position.x, amcl_msg.pose.pose.position.y, yaw + np.pi / 2]  # Convert to radians and adjust orientation

print(process_goal(goal, pose, cartesian=True))
print(process_goal(goal, pose), pose[-1] * 180 / np.pi)

### 5. GT trajectories

In [ ]:
import time

t0 = time.time()
for _ in range(100):
    with open('/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/paths/1008_0.pkl', 'rb') as f:
        data = pickle.load(f)

tf1 = (time.time() - t0) /100
print("Original time: ", tf1)

t0 = time.time()
for _ in range(100):
    with open('/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/bb/1008_0.pkl', 'rb') as f:
        data = pickle.load(f)

tf2 = (time.time() - t0) /100
print("New time: ", tf2)

print("perceptual increase: ", tf2 / tf1)


In [ ]:
import pickle
with open('/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/paths/1008_0.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())

plt.imshow(data['camera'][0])
plt.imshow(data['local_map'])
plt.plot(data['path'][:, 1]/0.1+300, data['path'][:, 0]/0.1+300, 'r-')


print(data['all_paths'][0].shape)

plt.scatter(data['lidar'][0][:, 1]/0.1+300, data['lidar'][0][:, 0]/0.1+300, s=1, c='b', label='Scan Points')
plt.ylim(600, 0)
plt.xlim(0, 600)

leng = 0
for i in range(len(data['path']) - 1):
    leng += np.linalg.norm(data['path'][i] - data['path'][i + 1])
print(leng)

In [ ]:
def rotate_pointcloud(pcl, theta_rad):
    # Rotation matrix around Z
    R = np.array([
        [np.cos(theta_rad), -np.sin(theta_rad), 0],
        [np.sin(theta_rad),  np.cos(theta_rad), 0],
        [0,                  0,                 1]
    ])
    return pcl @ R.T  # Rotate each point

In [ ]:
import pickle
with open('/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/bb/592_0.pkl', 'rb') as f:
    data = pickle.load(f)

print(data['scan'])

# plt.imshow(data['camera'][0])
plt.imshow(data['local_map'])
plt.plot(data['path'][:, 1]/0.1+300, data['path'][:, 0]/0.1+300, 'r-')

pcl = data['lidar_dn'][0]
print(pcl.shape)
print(data['goal'])
mask = (pcl[:, 2] < 2) & (pcl[:, 2] > 0.18) #& (np.linalg.norm(pcl, axis=1) < 30)
# pcl = pcl[mask]

for path in data['all_paths']:
    # path = world_to_local(path, data['pose'])
    plt.plot(path[:, 1]/0.1+300, path[:, 0]/0.1+300, 'g-')

plt.scatter(pcl[:, 1]/0.1+300, pcl[:, 0]/0.1+300, s=1, c='b', label='Scan Points')
# plt.ylim(600, 0)
# plt.xlim(0, 600)

leng = 0
for i in range(len(data['path']) - 1):
    leng += np.linalg.norm(data['path'][i] - data['path'][i + 1])
print(data['pose'])

In [ ]:
local_map = data['local_map']

mask1 = (pcl[:, 2] < 1.5) #& (pcl[:, 2] > 0.18) & (np.linalg.norm(pcl, axis=1) < 30)

pcl_img = np.clip(pcl / 0.1 + 300, 0, 600).astype(np.int64)
pcl_x = pcl_img[:, 1]
pcl_y = pcl_img[:, 0]

mask = (pcl_x >= 0) & (pcl_x < 600) & \
       (pcl_y >= 0) & (pcl_y < 600)

pcl_y = pcl_y[mask]
pcl_x = pcl_x[mask]

im = np.zeros((600, 600), dtype=np.uint8)

im[pcl_y, pcl_x] = 255 * (local_map[pcl_y, pcl_x])
im[pcl_y, pcl_x] = 127 * (1 - local_map[pcl_y, pcl_x])

colors = data['local_map'][pcl_y, pcl_x]
plt.imshow(im)
plt.imshow(data['local_map'], alpha=0.1)

# categories = ['A', 'B', 'C']
# colors = {'A': 'red', 'B': 'green', 'C': 'blue'}
# for category in categories:
    # plt.scatter(x[mask], y[mask], c=colors[category], label=category)
# plt.legend()

In [ ]:
import numpy as np
import open3d as o3d
from PIL import Image

def create_multi_class_bev(pcl, semantic_map, resolution=0.1, output_size=(600, 600)):
    """
    Create BEV image with multiple semantic classes and export colored PCD
    
    Args:
        pcl: Point cloud (N,3) [x,y,z]
        semantic_map: 2D numpy array with class indices (0=background, 1=class1, ...)
        resolution: meters per pixel
        output_size: (width, height) of output image
    
    Returns:
        bev_image: RGB BEV image (H,W,3)
        colored_pcl: Point cloud with class colors (N,6) [x,y,z,r,g,b]
    """
    # Define color palette (extend as needed)
    class_colors = {
        0: [0, 255, 0],     # Background (black)
        1: [255, 0, 0],     # Class 1 (red)
        2: [0, 255, 0],     # Class 2 (green)
        3: [0, 0, 255],     # Class 3 (blue)
        # Add more classes as needed
    }
    
    # Convert point cloud to pixel coordinates
    bev_width, bev_height = output_size
    pcl_img = np.clip(pcl / resolution + np.array([bev_width/2, bev_height/2, 0]), 0, max(bev_width, bev_height)-1).astype(np.int64)
    pcl_x = pcl_img[:, 1]  # BEV image x-coordinate (matrix column)
    pcl_y = pcl_img[:, 0]  # BEV image y-coordinate (matrix row)
    
    # Filter points within image bounds
    mask = (pcl_x >= 0) & (pcl_x < bev_width) & \
           (pcl_y >= 0) & (pcl_y < bev_height)
    pcl_x = pcl_x[mask]
    pcl_y = pcl_y[mask]
    filtered_pcl = pcl[mask]
    
    # Create colored BEV image
    bev_image = np.zeros((bev_height, bev_width, 3), dtype=np.uint8)
    
    # Get class indices for each point
    point_classes = semantic_map[pcl_y, pcl_x]
    
    # Color each point according to its class
    conditions = [True, True]#filtered_pcl[:, 2] < 0.0]

    for (class_idx, color) in class_colors.items():
        class_mask = point_classes == class_idx
        bev_image[pcl_y[class_mask], pcl_x[class_mask]] = color
    
    # Create colored point cloud for PCD export
    colored_pcl = np.zeros((filtered_pcl.shape[0], 6))  # x,y,z,r,g,b
    colored_pcl[:, :3] = filtered_pcl
    for (class_idx, color) in class_colors.items():
        class_mask = point_classes == class_idx
        colored_pcl[class_mask, 3:] = color
        colored_pcl[colored_pcl[:, 2] > 1.0, 3:] = class_colors[1]

    return bev_image, colored_pcl


def save_colored_pcd(pcl_with_colors, filename):
    """
    Save colored point cloud to PCD file
    
    Args:
        pcl_with_colors: (N,6) array [x,y,z,r,g,b]
        filename: Output PCD filename
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl_with_colors[:, :3])
    pcd.colors = o3d.utility.Vector3dVector(pcl_with_colors[:, 3:] / 255.0)
    o3d.io.write_point_cloud(filename, pcd)

In [ ]:
pcl = data['lidar_dn'][2]
semantic_map = data['local_map']  # Random classes 0-3

# Generate BEV and colored point cloud
bev_image, colored_pcl = create_multi_class_bev(
    pcl[:, :3], 
    semantic_map,
    resolution=0.1,
    output_size=(600, 600)
)

# Save results
# Image.fromarray(bev_image).save("multi_class_bev.png")
save_colored_pcd(colored_pcl, "colored_cloud.pcd")

plt.imshow(bev_image)

# plt.imshow(data['camera'][0])

### que es lo que falta realmente:

1. generar todos los pkl sin gt (para todos los cosos). kinda done.
2. generar todos los gt a partir de los pkl
3. mergear los datos. Nuevo modo de la funcion original para esto.
4. sfijarse bien en los x, y
5 entrenar mierda.


In [ ]:
import pickle

file_path = '/home/tesistas/Desktop/GONZALO/gnd_dataset/local_map_files_120/data.pkl'
with open(file_path, 'rb') as f:
        data = pickle.load(f)

print(data)